In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time


In [ ]:
#Used headers/agent because the request was timed out and asking for an agent. 
#Using following code we can fake the agent.
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
response = requests.get("https://www.lahaus.mx/venta/propiedades/area-metropolitana-monterrey",headers=headers)
content = response.content
soup = BeautifulSoup(content,"html.parser")

In [50]:
def create_newrow(cols_list,data_list):
       
    newrow_dict = {}
    for i,j in zip(cols_list,data_list):
        newrow_dict[i] = j
        
    return newrow_dict

def merge_df(newrow_dict,target_df):
    
    newrow_df = pd.DataFrame(newrow_dict, index=[len(target_df)])
    target_df = pd.concat([target_df,newrow_df])
    
    return target_df

def get_pages(soup):
       
    p_num = 0
    nav = soup.find("nav",attrs={'id':'srp-pagination'})
    for i,page in enumerate(nav.find_all("li",attrs={'class':''})):
        if (page.find("a")) is not None:
            p_num = int(page.find("a").text)

    return p_num

def get_soup(i,wt):
    
    print("Retrieving content from page: ",i+1)
    
    webp = 'https://www.lahaus.mx/venta/propiedades/area-metropolitana-monterrey'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    
    if i == 0:
        response = requests.get(webp,headers=headers)
    else:
        response = requests.get(webp + '?pagina=' + str(i+1),headers=headers)
    
    content = response.content
    soup = BeautifulSoup(content,"html.parser")
    time.sleep(wt)  
    
    return(soup)


def get_categories(res,cat):
    
    temp_list = []
    
    jungle_green = res.find("span",attrs={'class':'leading-8 text-12 lg:text-10 xl:text-12 rounded-sm text-lh-jungle-green bg-lh-caribbean-green px-lh-8 py-lh-4 mr-5 truncate'})
    green_gray = res.find("span",attrs={'class':'leading-10 text-12 lg:text-10 xl:text-12 border rounded-sm text-lh-green-gray border-lh-gray px-lh-8 mr-5 truncate'})
    used_green = res.find("span",attrs={'class':'leading-8 text-12 lg:text-10 xl:text-12 rounded-sm text-success-800 bg-success-50 px-lh-8 py-lh-4 mr-5 truncate'})
    yellow = res.find("span",attrs={'class':'leading-8 text-12 lg:text-10 xl:text-12 rounded-sm text-lh-jungle-green bg-lh-yellow px-lh-8 py-lh-4 mr-5'})
    
    if (jungle_green) is not None:
        temp_list.append(jungle_green.text)
        cat = 1
    if (used_green):
        temp_list.append(used_green.text)
        cat = 2
    if (yellow):
        temp_list.append(yellow.text)
    if (green_gray) is not None:
        temp_list.append(green_gray.text)
    
    return temp_list,cat

In [114]:
df_lahaus = pd.DataFrame(columns = ['loc','name','desc','ammenities','categories','price_range','unit','unit_area','unit_price','bedrooms','bathrooms','parking_spots','comments','date'])

df_lahaus = build_df(df_lahaus,5)

Retrieving content from page:  1
Current record count: 50
Completed pass number 1 of 1
Retrieving content from page:  2
Current record count: 100
Completed pass number 2 of 1
Retrieving content from page:  3
Current record count: 150
Current record count: 200
Completed pass number 3 of 1
Retrieving content from page:  4
Current record count: 250
Completed pass number 4 of 1
Retrieving content from page:  5
Completed pass number 5 of 1
Retrieving content from page:  6
Current record count: 300
Completed pass number 6 of 1
Retrieving content from page:  7
Completed pass number 7 of 1
Retrieving content from page:  8
Current record count: 350
Completed pass number 8 of 1
Retrieving content from page:  9
Completed pass number 9 of 1
Retrieving content from page:  10
Completed pass number 10 of 1
Retrieving content from page:  11
Current record count: 400
Completed pass number 11 of 1
Retrieving content from page:  12
Completed pass number 12 of 1
Retrieving content from page:  13
Current r

In [116]:
def build_df(df,wt):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    response = requests.get("https://www.lahaus.mx/venta/propiedades/area-metropolitana-monterrey",headers=headers)
    content = response.content
    soup = BeautifulSoup(content,"html.parser")
    
    num_pages = get_pages(soup)
                
    for i in range(num_pages):
        
        soup = get_soup(i,wt)
        
        #lahaus_res = soup.find_all("article",attrs={"class":"property-card-container border mb-lh-24 md:flex-row rounded-lg w-full cursor-pointer new-property border-lh-md-gray"})
        lahaus_res = soup.find_all("article",attrs={"class":"property-card-container"})

        for res in lahaus_res:
            cat = 0
            categories = []
            
            categories,cat = get_categories(res,cat)
            categories_con = "; ".join([str(k).strip() for k in categories])
            date = datetime.date.today()

            if cat == 1:
                
                name = (res.find("a").contents[0]).strip()
                loc = (res.find("span",attrs={'class':'label-r-12 order-2 text-lh-green-gray mt-6 mb-4'}).text).strip()
                desc = (res.find("p",attrs={'class':'order-4 text-12 font-regular text-lh-dark-deep-blue mb-3 leading-7'}).text).strip()
                ammenities = (res.find("span",attrs={'class':'label-r-12 order-5 text-lh-green-gray mb-8'}).text).strip()
                price_range = (res.find("p",attrs={'class':'order-6 text-14 font-bold text-lh-dark-deep-blue'}).text).strip()
                
                for u,unit in enumerate(res.find_all("li", attrs={'class':'p-lh-12 mr-lh-12 border rounded-lg border-lh-md-gray hover:bg-lh-extra-light-blue'})):

                    unit_area = (unit.find("p",attrs={'class':'text-14 text-lh-dark-deep-blue text-lh-green-gray'}).text).strip()
                    unit_price = (unit.find("p",attrs={'class':'small-r-14 text-lh-jungle-green mt-4'}).text).strip()

                    for details in unit.find_all("span",attrs={'class':'mr-lh-16'}):                
                        
                        #print(details)
                        if (details.find("i",attrs={'class':'fa fa-real-estate-bedroom'})) is not None:
                            bedrooms = (details.text).strip()
                        if (details.find("i",attrs={'class':'fa fa-real-estate-bathroom'})) is not None:
                            bathrooms = (details.text).strip()
                        if (details.find("i",attrs={'class':'fa fa-real-estate-parking'})) is not None:
                            parking_spots = (details.text).strip()
                        else:
                            parking_spots = 0
                    
                    comments = ''
                    #print([loc,name,desc,ammenities,categories_con,price_range,u+1,unit_area,unit_price,bedrooms,bathrooms,parking_spots])
                    list_cols = ['loc','name','desc','ammenities','categories','price_range','unit','unit_area','unit_price','bedrooms','bathrooms','parking_spots','comments','date']
                    newrow_list = [loc,name,desc,ammenities,categories_con,price_range,len(df),unit_area,unit_price,bedrooms,bathrooms,parking_spots,comments,date]            

                    df = merge_df(create_newrow(list_cols,newrow_list),df)
                    if len(df) % 50 == 0:
                        print("Current record count: {}".format(len(df)))
                        
            elif cat == 2:
                
                desc = (res.find("a").contents[0]).strip()
                #print(desc)
                loc = (res.find("span",attrs={'class':'label-r-12 order-2 text-lh-green-gray mt-lh-16 mb-lh-8 mx-lh-16'}).text).strip()
                ammenities = (res.find("span",attrs={'class':'order-4 label-r-12 order-2 text-lh-green-gray mx-lh-16'}).text).strip()
                if (res.find("p",attrs={'class':'order-4 text-14 text-lh-dark-deep-blue mt-lh-24 mb-lh-32 ml-lh-16 lh-font-weight-4'})) is not None:                 
                    price_range = (res.find("p",attrs={'class':'order-4 text-14 text-lh-dark-deep-blue mt-lh-24 mb-lh-32 ml-lh-16 lh-font-weight-4'}).text).strip()
                else:
                    price_range = (res.find("span",attrs={'class':'text-success-800'}).text).strip()
                
                for attr in res.find_all("span",attrs = {'class':'mr-lh-24 lg:mr-lh-12 xl:mr-lh-24 text-lh-green-gray'}):
                    
                    used_details = attr.find("span",attrs={'class':'mr-lh-24 lg:mr-lh-12 xl:mr-lh-24 text-lh-green-gray'})
                    #print(attr)
                    
                    if attr.find("i",attrs={'class':'fa fa-real-estate-apartment color-silver'}) is not None:
                        comments = (attr.text).strip()
                    if attr.find("i",attrs={'class':'fa fa-real-estate-bedroom color-silver'}) is not None:
                        bedrooms = (attr.text).strip()
                    if attr.find("i",attrs={'class':'fa fa-real-estate-bathroom color-silver'}) is not None:
                        bathrooms = (attr.text).strip()
                    if attr.find("i",attrs={'class':'fa fa-area-outline color-silver'}) is not None:
                        unit_area = (attr.text).strip()
                    if attr.find("i",attrs={'class':'fa fa-real-estate-parking color-silver'}) is not None:
                        parking_spots = (attr.text).strip()
                    else:
                        parking_spots = 0
                    
                list_cols = ['loc','name','desc','ammenities','categories','price_range','unit','unit_area','unit_price','bedrooms','bathrooms','parking_spots','comments','date']
                newrow_list = [loc,'Propiedad Usada',desc,ammenities,categories_con,price_range,len(df),unit_area,'',bedrooms,bathrooms,parking_spots,comments,date]            

                df = merge_df(create_newrow(list_cols,newrow_list),df)
                if len(df) % 50 == 0:
                    print("Current record count: {}".format(len(df)))
                
            else:
                
                loc = (res.find("p",attrs={'class':'order-3 text-14 font-regular text-lh-green-gray mb-lh-16'}).text).strip()
                price_range = (res.find("p",attrs={'class':'order-4 text-18 font-regular text-lh-jungle-green'}).text).strip()
                
                list_cols = ['loc','name','desc','ammenities','categories','price_range','unit','unit_area','unit_price','bedrooms','bathrooms','parking_spots','comments','date']
                newrow_list = [loc,name,'Lanzamiento LA HAUS','',categories_con,price_range,len(df),'',unit_price,'',bathrooms,'','',date]
                
                
        print("Completed pass number {} of {}".format(i+1,num_pages))
        
    print("Job ended")
    return df


In [117]:
df_lahaus

,loc,name,desc,ammenities,categories,price_range,unit,unit_area,unit_price,bedrooms,bathrooms,parking_spots,comments,date
0,"Monterrey, Área Metropolitana Monterrey",Semillero Mediterráneo,Semillero Mediterráneo es tú mejor inversión. ...,Incluye: Zona de Mascotas · Espacio Yoga · Can...,Desarrollo nuevo; Preventa,De 1.96 a $2.63 millones,0,59 ㎡,$2.63 millones,2,2,1,,2022-09-28
1,"Monterrey, Área Metropolitana Monterrey",Semillero Mediterráneo,Semillero Mediterráneo es tú mejor inversión. ...,Incluye: Zona de Mascotas · Espacio Yoga · Can...,Desarrollo nuevo; Preventa,De 1.96 a $2.63 millones,1,49 ㎡,$2.61 millones,1,1,1,,2022-09-28
2,"Monterrey, Área Metropolitana Monterrey",Semillero Mediterráneo,Semillero Mediterráneo es tú mejor inversión. ...,Incluye: Zona de Mascotas · Espacio Yoga · Can...,Desarrollo nuevo; Preventa,De 1.96 a $2.63 millones,2,50 ㎡,$2.48 millones,2,1,1,,2022-09-28
3,"Monterrey, Área Metropolitana Monterrey",Semillero Mediterráneo,Semillero Mediterráneo es tú mejor inversión. ...,Incluye: Zona de Mascotas · Espacio Yoga · Can...,Desarrollo nuevo; Preventa,De 1.96 a $2.63 millones,3,48 ㎡,$2.44 millones,2,1,1,,2022-09-28
4,"Monterrey, Área Metropolitana Monterrey",Semillero Mediterráneo,Semillero Mediterráneo es tú mejor inversión. ...,Incluye: Zona de Mascotas · Espacio Yoga · Can...,Desarrollo nuevo; Preventa,De 1.96 a $2.63 millones,4,41 ㎡,$2.3 millones,1,1,1,,2022-09-28
5,"Monterrey, Área Metropolitana Monterrey",Amairé,"AMAIRÉ combina diversidad, intimidad y natural...",Incluye: Gimnasio al Aire Libre · Juegos Infan...,Desarrollo nuevo; Entrega Inmediata,De 2.1 a $4.59 millones,5,120 ㎡,$4.59 millones,3,3,1,,2022-09-28
6,"Monterrey, Área Metropolitana Monterrey",Amairé,"AMAIRÉ combina diversidad, intimidad y natural...",Incluye: Gimnasio al Aire Libre · Juegos Infan...,Desarrollo nuevo; Entrega Inmediata,De 2.1 a $4.59 millones,6,100 ㎡,$3.88 millones,2,2,1,,2022-09-28
7,"Monterrey, Área Metropolitana Monterrey",Amairé,"AMAIRÉ combina diversidad, intimidad y natural...",Incluye: Gimnasio al Aire Libre · Juegos Infan...,Desarrollo nuevo; Entrega Inmediata,De 2.1 a $4.59 millones,7,80 ㎡,$3.44 millones,2,1,1,,2022-09-28
8,"Monterrey, Área Metropolitana Monterrey",Amairé,"AMAIRÉ combina diversidad, intimidad y natural...",Incluye: Gimnasio al Aire Libre · Juegos Infan...,Desarrollo nuevo; Entrega Inmediata,De 2.1 a $4.59 millones,8,40 ㎡,$2.1 millones,1,1,1,,2022-09-28
9,"Guadalupe, Área Metropolitana Monterrey",Privadas Masai,Privadas Masái es un exclusivo desarrollo con ...,Incluye: Cancha de Fútbol · Juegos Infantiles ...,Casa nueva; Entrega Inmediata,De 1.62 a $1.84 millones,9,148 ㎡,$1.84 millones,3,3,1,,2022-09-28


In [118]:
pd.reset_option('all')

C:\Users\norbe\anaconda3\lib\site-packages\pandas\_config\config.py:653: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  warnings.warn(d.msg, FutureWarning)
C:\Users\norbe\anaconda3\lib\site-packages\pandas\_config\config.py:653: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)
